# "Uploading" PDFs to Claude Via the API

One really nice feature of [Claude.ai](https://www.claude.ai) is the ability to upload PDFs. Let's mock up that feature in a notebook, and then test it out by summarizing a long PDF.

In [5]:
#!pip install pypdf
!pip install boto3 botocore --upgrade

DEPRECATION: Loading egg at /home/alfred/anaconda3/envs/llm/lib/python3.11/site-packages/flash_attn-2.0.7-py3.11-linux-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /home/alfred/anaconda3/envs/llm/lib/python3.11/site-packages/ninja-1.11.1-py3.11-linux-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /home/alfred/anaconda3/envs/llm/lib/python3.11/site-packages/einops-0.7.0rc2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/91/a9/c6eea6c80c57d8957eaba3cc1122339a1adbdce60532cbf94046c312bcd3/boto3-1.28.57-py3-none-any.whl.metadata
  Using cached boto3-1.28.57-py3-none-any.whl.metadata (6.7

In [1]:
!curl -O https://arxiv.org/pdf/2212.08073.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2039k  100 2039k    0     0  3301k      0 --:--:-- --:--:-- --:--:-- 3304k


Now, we'll use pypdf to read the pdf. It's not identical to what Claude.ai uses behind the scenes, but it's pretty close.

In [2]:
from pypdf import PdfReader

reader = PdfReader("2212.08073.pdf")
number_of_pages = len(reader.pages)
text = ''.join([page.extract_text() for page in reader.pages])
print(text[:2155])

Constitutional AI: Harmlessness from AI Feedback
Yuntao Bai∗, Saurav Kadavath, Sandipan Kundu, Amanda Askell, Jackson Kernion,
Andy Jones, Anna Chen, Anna Goldie, Azalia Mirhoseini, Cameron McKinnon,
Carol Chen, Catherine Olsson, Christopher Olah, Danny Hernandez, Dawn Drain,
Deep Ganguli, Dustin Li, Eli Tran-Johnson, Ethan Perez, Jamie Kerr, Jared Mueller,
Jeffrey Ladish, Joshua Landau, Kamal Ndousse, Kamile Lukosuite, Liane Lovitt,
Michael Sellitto, Nelson Elhage, Nicholas Schiefer, Noemi Mercado, Nova DasSarma,
Robert Lasenby, Robin Larson, Sam Ringer, Scott Johnston, Shauna Kravec,
Sheer El Showk, Stanislav Fort, Tamera Lanham, Timothy Telleen-Lawton, Tom Conerly,
Tom Henighan, Tristan Hume, Samuel R. Bowman, Zac Hatﬁeld-Dodds, Ben Mann,
Dario Amodei, Nicholas Joseph, Sam McCandlish, Tom Brown, Jared Kaplan∗
Anthropic
Abstract
As AI systems become more capable, we would like to enlist their help to supervise
other AIs. We experiment with methods for training a harmless AI assistant

In [1]:
import boto3
import json
## Older version boto3
'''
bedrock = boto3.client(
    service_name="bedrock",
    region_name="us-east-1",
    endpoint_url="https://bedrock.us-east-1.amazonaws.com",
)
bedrock.list_foundation_models()
'''
## Use this for boto3 1.28.57
br_client = boto3.client('bedrock')
br_runtime_client = boto3.client('bedrock-runtime')
br_client.list_foundation_models()

UnknownServiceError: Unknown service: 'bedrock-runtime'. Valid service names are: accessanalyzer, account, acm, acm-pca, alexaforbusiness, amp, amplify, amplifybackend, amplifyuibuilder, apigateway, apigatewaymanagementapi, apigatewayv2, appconfig, appconfigdata, appfabric, appflow, appintegrations, application-autoscaling, application-insights, applicationcostprofiler, appmesh, apprunner, appstream, appsync, arc-zonal-shift, athena, auditmanager, autoscaling, autoscaling-plans, backup, backup-gateway, backupstorage, batch, bedrock, billingconductor, braket, budgets, ce, chime, chime-sdk-identity, chime-sdk-media-pipelines, chime-sdk-meetings, chime-sdk-messaging, chime-sdk-voice, cleanrooms, cloud9, cloudcontrol, clouddirectory, cloudformation, cloudfront, cloudhsm, cloudhsmv2, cloudsearch, cloudsearchdomain, cloudtrail, cloudtrail-data, cloudwatch, codeartifact, codebuild, codecatalyst, codecommit, codedeploy, codeguru-reviewer, codeguru-security, codeguruprofiler, codepipeline, code

With the paper downloaded and in memory, we can ask Claude to perform various fun tasks with it.

In [4]:
prompt = f"""\n\nHuman: Here is an academic paper: <paper>{text}</paper>

Please do the following:
1. Summarize the abstract at a kindergarten reading level. (In <kindergarten_abstract> tags.)
2. Write the Methods section as a recipe from the Moosewood Cookbook. (In <moosewood_methods> tags.)
3. Compose a short poem epistolizing the results in the style of Homer. (In <homer_results> tags.)
4. Write a grouchy critique of the paper from a wizened PI. (In <grouchy_critique> tags.)

Assistant:"""

In [5]:
len(prompt)

119161

In [7]:
body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 8000})
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = br_runtime_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)

In [8]:
from pprint import pprint
response_body = json.loads(response.get("body").read())
pprint(response_body.get("completion"))

(' <kindergarten_abstract>\n'
 'The scientists wanted to teach a robot to be nice. They gave it some simple '
 'rules about being good. Then they had it look at what it said and fix it if '
 "it wasn't nice. After, they let it practice being nice by talking to people. "
 'Now the robot is nicer!\n'
 '</kindergarten_abstract>\n'
 '\n'
 '<moosewood_methods>\n'
 'Constitutional AI\n'
 '\n'
 'Ingredients:\n'
 '- 1 helpful robot \n'
 '- 10 rules for being nice\n'
 '- 100 mean questions to ask the robot\n'
 '- 1 recipe for fixing mean things the robot says\n'
 '\n'
 'Instructions:\n'
 "1. Start with a robot that wants to be helpful. We'll call it Robot A.  \n"
 '2. Give Robot A some mean questions that might make it say bad things.\n'
 '3. Listen to what Robot A says back. Uh oh, some of those things are not '
 'nice!\n'
 '4. Show Robot A the 10 rules for being nice. Tell it to look at what it just '
 'said and fix anything that breaks the rules.\n'
 '5. Robot A will try to make what it said

## Call Anthropic API directly

In [ ]:
import anthropic
API_KEY = os.environ['ANTHROPIC_API_KEY']
CLIENT = anthropic.Client(api_key=API_KEY)
def get_completion(client, prompt, max_tokens=3000, model='claude-2'):
    return client.completions.create(
        prompt=prompt, max_tokens_to_sample=max_tokens, model=model
    ).completion

In [25]:
completion = get_completion(CLIENT,
    f"""\n\nHuman: Here is an academic paper: <paper>{text}</paper>

Please do the following:
1. Summarize the abstract at a kindergarten reading level. (In <kindergarten_abstract> tags.)
2. Write the Methods section as a recipe from the Moosewood Cookbook. (In <moosewood_methods> tags.)
3. Compose a short poem epistolizing the results in the style of Homer. (In <homer_results> tags.)
4. Write a grouchy critique of the paper from a wizened PI. (In <grouchy_critique> tags.)

Assistant:"""
)
print(completion)

<kindergarten_abstract>
The scientists wanted to teach a robot to be nice and not say mean things. They didn't want to tell it everything it should say, they wanted it to learn by itself. 

First they let it try talking to people. When it said bad things, they told it to explain why that was bad. Then they had it try to say it nicer. They did this over and over to teach it.

After, they let it talk to itself about which thing it said was more nice. Then they used that to teach it more.

In the end, the robot could talk nicely without the scientists having to tell it every word.
</kindergarten_abstract>

<moosewood_methods>

Teaching a Robot to Be Nice

Ingredients:
- 1 large helpful robot
- 1 cup red teaming prompts (designed to elicit harmful responses)
- 1/4 cup principles of ethics 
- 2 tbsp few-shot examples

Instructions:

1. Let the robot sample responses to the red teaming prompts. These will likely be harmful. 

2. Have the robot critique its own responses according to the prin